In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
sys.path.append('/home/manu/TFG_repo/scripts')  # make modules in scripts folder callable from notebooks

In [59]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from scipy.io import loadmat
from processing_routines import datenum_to_epoch
import netCDF4

In [6]:
from config import data_dir
from gen_time_series

In [29]:
sensor_data = loadmat(data_dir / 'raw/preassure_sensor' / 'DanoisMoat-439923N-AQ1061-bott1092-20180818-20190829.mat')

In [99]:
save_path = data_dir / 'time_series' / 'pressure_sensor' / '20181116_DanoisMoat.csv'

In [162]:
datenum = np.squeeze(sensor_data['dates'])
pres = np.squeeze(sensor_data['pre'])
temp = np.squeeze(sensor_data['tem'])
vel = np.squeeze(sensor_data['vel'])

In [31]:
epoch = datenum_to_epoch(datenum)

In [32]:
datetime64 = np.array(epoch, dtype='datetime64[s]')

In [52]:
i_0 = np.where(datetime64 == np.datetime64('2018-11-16T11'))[0][0]
i_f = np.where(datetime64 == np.datetime64('2019-04-08T11'))[0][0]

In [167]:
epoch_series = epoch[i_0:i_f+1]
datetime64_series = datetime64[i_0:i_f+1]
pres_series = pres[i_0:i_f+1]
temp_series = temp[i_0:i_f+1]
vel_series = vel[i_0:i_f+1]

In [67]:
dim_time = len(epoch_series)
with netCDF4.Dataset(save_path, mode='w', format='NETCDF4') as ds:
        # dimensions
        ds.createDimension('date', dim_time)
        
        # variables
        date_ = ds.createVariable('date', 'i4', ('date',))
        pres_ = ds.createVariable('pres', 'f8', ('date',))

        date_[:] = epoch_series
        pres_[:] = pres_series

In [168]:
df = pd.DataFrame(zip(datetime64_series, pres_series, temp_series, vel), columns=['date', 'pres', 'temp', 'vel'])

In [169]:
df

,date,pres,temp,vel
0,2018-11-16 11:00:00,1061.731,9.789,3.2
1,2018-11-16 11:15:00,1061.704,9.800,6.6
2,2018-11-16 11:30:00,1061.632,9.800,8.0
3,2018-11-16 11:45:00,1061.585,9.788,6.2
4,2018-11-16 12:00:00,1061.469,9.763,4.2
...,...,...,...,...
13724,2019-04-08 10:00:00,1059.649,9.702,2.2
13725,2019-04-08 10:15:00,1059.527,9.693,4.3
13726,2019-04-08 10:30:00,1059.416,9.702,4.9
13727,2019-04-08 10:45:00,1059.389,9.698,4.0


In [171]:
df.to_csv(save_path, index=False)